# L'algorithme de descente de gradient 

## 1. Le cas d'une fonctionnelle quadratique
 
Tout d'abord, nous appliquons la méthode à une fonctionnelle quadratique :
\begin{align}\label{quadf}\tag{1} f(x)=\dfrac12 x^TA x + b^T x +c,\qquad \text{pour } x\in \mathbb{R}^ N, \end{align}
avec
$A$ une matrice réelle $N\times N$, symétrique définie positive, $b\in\mathbb{R}^N$ et $c\in \mathbb{R}$. 

**Question 1.** Calculez le gradient et la matrice Hessienne de $f$.

**Question 2.** Que pouvez-vous en déduire sur la nature de $f$ ?

**Question 3.** Montrez que $f$ atteint son minimum sur $\mathbb{R}^N$ en un seul point $x^*$. Donnez une caractérisation de ce point. 

### 1.1 L'algorithme de descente de gradient à pas optimal

Soit $f$ une fonction convexe et coercive de classe $C^1$ sur $\mathbb{R}^N$. L'algorithme optimal de descente de gradient est défini comme suit. 

Soit $x^0\in \mathbb{R}^N$ (on essaie de choisir $x^0$ proche de $x^*$, en l'absence d'indication alors on prend $x^0=0$). 

Ensuite, pour $k=0,1,2,\ldots\ $ jusqu'à convergence, répéter : 

$$
\left|
\begin{array}{lcl}
d^k& \longleftarrow & -\nabla f(x^k),\\
\tau^k &\longleftarrow &\mathop{argmin}_{t>0} f(x^k + td^k),\\ 
x^{k+1}&\longleftarrow &x^k+\tau^k d^k
\end{array}
\right.
$$

**Question 4.** Proposez un critère d'arrêt pour l'algorithme qui utilise la caractérisation de la question **3**. 

*Remarque :** En général, on ne sait pas calculer $\tau^k$ et en pratique, la deuxième étape est remplacée par une recherche approchée. Cependant, lorsque $f$ est quadratique, le calcul de $\tau^k$ est facile. 

**Question 5.** Dans le cas de la fonction quadratique (1), explicitez $d^k$ et $\tau^k$ comme fonctions de $A$, $x^k$ et $b$. 

Maintenant, nous spécifions $N=2$ et
$$ A=\binom{C\quad 0}{0\quad 1},\quad C\ge 1,\quad b=0,\quad c=0.$$
**Question 6.** Quel est l'infimum de $f$ sur $\mathbb{R}^2$ dans ce cas ? Donner $x^*$. 

**Question 7.** Deux fonctions sont données ci-dessous :
- une fonction qui dessine un champ de vecteur donné par une application $F$. À titre d'exemple, il est appliqué à l'application $G(x,y)=(x, 25y)$.
- une fonction qui dessine quelques lignes de niveau d'une fonction $f$. Il est appliqué à $g(x,y)=\dfrac{x^2+25y^2}2$. Notez que $G=\nabla g$. Qu'observez-vous ? 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

def draw_vector_field(F, xmin, xmax, ymin, ymax, N=15):
    X = np.linspace(xmin, xmax, N)  # x coordinates of the grid points
    Y = np.linspace(ymin, ymax, N)  # y coordinates of the grid points
    U, V = F(*np.meshgrid(X, Y))  # vector field
    M = np.hypot(U, V)  # compute the norm of (U,V)
    M[M == 0] = 1  # avoid division by 0
    U /= M  # normalize the u componant
    V /= M  # normalize the v componant
    return plt.quiver(X, Y, U, V, angles='xy')

def level_lines(f, xmin, xmax, ymin, ymax, levels, N=500):
    x = np.linspace(xmin, xmax, N)
    y = np.linspace(ymin, ymax, N)
    z = f(*np.meshgrid(x, y))
    level_l = plt.contour(x, y, z, levels=levels)
    #plt.clabel(level_l, levels, fmt='%.1f') 


g = lambda x, y: .5*(x**2 + 12*y**2)
G = lambda x, y: np.array([x, 12*y])
%matplotlib inline
plt.figure(figsize=(12,6))
level_lines(g, -8, 8, -2.1, 2.1, np.linspace(0, 25, 5))
draw_vector_field(G,  -8, 8, -2.1, 2.1, 10)
plt.axis('equal')
plt.show()

### Observations
* L'intensité des lignes de niveau et la direction du gradient montrent qu'un minimum global de g est atteint au point (0,0).
* On peut également observer que g croît beaucoup plus rapidement selon y que selon x (facteur 25), ce qui se traduit par des lignes de niveau "aplaties" selon x, et des flèches de gradient plus rapprochées selon y.

**Question 8.** Implémentez l'algorithme de descente de gradient à pas optimal. Le point initial doit être $x^0=\binom1C$.

**Question 9.** Sur le même graphique, représentez les itérations, quelques lignes de niveau de $f$ et le champ de vecteur normalisé $\dfrac {1}{|\nabla f|}\nabla f$. 

In [ ]:
# Question 8

import numpy as np
import matplotlib.pyplot as plt 

f = lambda x, C: .5*(C*x[0]**2 + x[1]**2)
F = lambda x, C: np.array([C*x[0], x[1]])
x0 = lambda C: np.array([1, C])

norm = lambda A: np.linalg.norm(A)
tauOpt = lambda x, C: (C**2 * x[0]**2 + x[1]**2) / (C**3 * x[0]**2 + x[1]**2)


def gradient_descent(C, tol=1e-8):
    fc = lambda x: f(x, C)
    Fc = lambda x: F(x, C)
    x = x0(C)
    epsilon = tol * norm(Fc(x0(C)))
    plt.plot(x[0], x[1], 'g*')
    counter = 0
    while(norm(Fc(x)) >= epsilon):
        counter += 1
        d = - Fc(x)
        tau = tauOpt(x,C)
        # tau = 1e-2
        x = x + tau*d
        plt.plot(x[0], x[1], 'ro')
    plt.plot(x[0], x[1], 'g')
    return x

In [ ]:
# Question 9

def plot_gradient(C) :
    f_plot = lambda x, y, C: C*x**2 + y**2
    F_plot = lambda x, y, C: np.array([2*C*x, 2*y])

    fc_plot = lambda x, y: f_plot(x, y, C)
    Fc_plot = lambda x, y: F_plot(x, y, C)

    %matplotlib inline
    plt.figure(figsize=(12,6))
    level_lines(fc_plot, -1*C, 1*C, -2*C, 2*C, np.linspace(0, 25, 15))
    draw_vector_field(Fc_plot,  -1*C, 1*C, -2*C, 2*C, 10)

    gradient_descent(C)

    plt.axis('equal')
    plt.show()
    
plot_gradient(1)

**Question 10.** Changer la valeur de $C$ de 1 à 32 ($C=1,2,4,8,16,32$). Qu'observez-vous ?

**Question 11.** Tracez le nombre d'itérations de la méthode en fonction de $C$. Faites une hypothèse. 

In [ ]:
# Question 10

Cs = [1, 2, 4, 8, 16, 32]
for C in Cs:
    plot_gradient(C)

In [ ]:
# Question 11

# Problème avec l'expression de tau

## 2. Le cas d'une fonction convexe régulière, line search. 

On considère la fonction définie par
$$
f(x,y):= \cosh(x) + \sin^2(x+y),\qquad \text{pour }z=(x,y)\in \mathbb{R}^2.
$$

**Question 12.** Montrer que $f$ est convexe au voisinage de $z^0_*$.

Nous allons appliquer un algorithme de descente de gradient avec ``line search'' à la fonction $f$. Plus précisément :

Étant donné  $z^0=(x^0,y^0)\in\mathbb{R}^2$, calculer de manière récursive, jusqu'à convergence,

$$
\left|
\begin{array}{lcl}
d^k& \longleftarrow & -\nabla f(z^k),\\
\tau^k &\longleftarrow & \text{Line-search}\ \left(\ t\mapsto f(z^k + td^k)\ \right),\\ 
z^{k+1}&\longleftarrow &z^k+\tau^k d^k
\end{array}
\right.
$$

Précisons la deuxième étape. On remarque d'abord que pour $t>0$,

$$
f(z^k+t d^k) \,=\, f(z^k) -t \|d^k\|^2 +o(t).
$$

En fait, si $f$ est convexe au voisinage de $z^k$, on a aussi pour $t>0$ assez petit, 

$$
f(z^k+t d^k)\, \ge\, f(z^k) -t \|d^k\|^2,
$$

donc on ne peut pas demander $f(z^k+t d^k) \,\le\, f(z^k) -t \|d^k\|^2$. 

L'idée de la *condition Armijo* est de demander un peu moins. Fixons un $\alpha\in (0,1)$ : la condition Armijo s'écrit : 

$$
\tag{2}f(z^k+t d^k)\, \le\, f(z^k) -\alpha\, t \|d^k\|^2.
$$

En utilisant le développement limité ci dessus, on a 

$$
\begin{array}{rcl}
f(z^k+t d^k) &=& f(z^k) -t \|d^k\|^2 +o(t)\\
   &=& f(z^k) -\alpha\, t \|d^k\|^2 - (1-\alpha)t\|d^k\|^2 +o(t)\\
   & = & f(z^k) -\alpha\, t \|d^k\|^2 -t \left[(1-\alpha)\|d^k\|^2 +o(1)\right]
\end{array}
$$

Pour $t>0$ assez petit, le terme entre crochet est positif et donc (2) est vrai.

Nous ne voulons pourtant pas choisir un $\tau^k$ trop petit (l'algorithme calerait). Pour éviter cela, nous fixons un pas maximal $\tau_0$ et un facteur $\beta\in(0,1)$ et nous testons successivement (2) avec $t=\tau_0$, $t=\tau_0\beta$, $t=\tau_0\beta^2$, ... 

On choisi $\tau^k=\tau_0\beta^j$ où $j$ est le premier entier tel que $t=\tau_0\beta^j$ vérifie (2).

Remarquez que comme $0<\beta<1$, cet entier existe. 

**Question 13.** Implémentez la méthode ci-dessus, avec $\alpha=0.5$, $\beta=0.75$. Commencez par $z^0=(1,0.5)$ et $\tau_0=1$. Ensuite, pour $k\ge 1$ utilisez $\tau_0\leftarrow\tau^0/\beta$.

Tout d'abord pour vous aider, la cellule suivante montre quelques ensembles de niveaux de $f$ et le champ de vecteur normalisé $\dfrac {1}{|\nabla f|}\nabla f$ au voisinage de $z^*$. 

In [ ]:
def draw_vector_field_2(F, xmin, xmax, ymin, ymax, N=15):
    X = np.linspace(xmin, xmax, N)  # x coordinates of the grid points
    Y = np.linspace(ymin, ymax, N)  # y coordinates of the grid points
    U, V = F(*np.meshgrid(X, Y))  # vector field
    M = np.hypot(U, V)  # compute the norm of (U,V)
    M[M == 0] = 1  # avoid division by 0
    U /= M  # normalize the u componant
    V /= M  # normalize the v componant
    return plt.quiver(X, Y, U, V, angles='xy')

def level_lines_2(f, xmin, xmax, ymin, ymax, levels, N=500):
    x = np.linspace(xmin, xmax, N)
    y = np.linspace(ymin, ymax, N)
    z = f(*np.meshgrid(x, y))
    level_l = plt.contour(x, y, z, levels=levels)
    #plt.clabel(level_l, levels, fmt='%.1f') 

f = lambda x, y : np.cosh(x)+ np.sin(x + y)**2
df = lambda x, y : np.array([np.sinh(x) + 2*np.cos(x + y)*np.sin(x + y), 2*np.cos(x + y)*np.sin(x + y)])
%matplotlib inline
plt.figure(figsize=(9,6))
level_lines_2(f, -1.1, 1.1, -1.1, 1.1, np.linspace(1, 3, 10))
draw_vector_field_2(df, -1.1, 1.1, -1.1, 1.1, 10)
plt.axis('equal')
plt.show()

In [ ]:
# Question 13

# ls -> line search
# alpha ?
def gradient_descent_ls(f, F, alpha, beta, z0, tau0, tol=1e-8, max_iterations = 100):
    counter = 0
    x = z0[0]
    y = z0[1]
    while(norm(F(x,y)) >= tol * norm(F(z0[0], z0[1])) and counter <= max_iterations):
        counter += 1
        d = -df(x,y)
        if(counter == 1): 
            tau = tau0
        else : tau = tau*beta
        x = x + tau*d[0]
        y = y + tau*d[1]
        plt.plot(x, y, 'ro')
    plt.plot(x, y, 'g')
    z = np.array([x,y])
    return z

In [ ]:
%matplotlib inline
plt.figure(figsize=(9,6))
level_lines_2(f, -1.1, 1.1, -1.1, 1.1, np.linspace(1, 3, 10))
draw_vector_field_2(df, -1.1, 1.1, -1.1, 1.1, 10)

gradient_descent_ls(f, df, 0.5, 0.75, np.array([0,0.5]), 1)

plt.axis('equal')
plt.show()

On considère maintenant la fonction définie sur $\mathbb{R}^3$ par 
$$
f(x,y,z):= \cosh(x) + \sin^2(x+y) + (y-z)^2,\qquad \text{pour }w=(x,y,z)\in \mathbb{R}^2.
$$

**Question 14.** Appliquez la méthode d'optimisation ci-dessus à cette fonction, en commençant par $w^0=(1,0.5,1)$. 

In [ ]:
# Pour les plots en 3 dimensions
from mpl_toolkits.mplot3d import Axes3D

# exemple
t = np.linspace(0,np.pi,101)
x, y, z = np.cos(t), np.sin(t), t+.5*np.sin(t)**2

ax = Axes3D(plt.figure())  # Define the 3D plot
ax.set(xlabel=r'$x$', ylabel=r'$y$', zlabel=r'$z$')
ax.plot(x, y, z,'.')  # Plot of the trajectory
plt.show()

In [ ]:
# Solution